In [ ]:
# -*- coding: utf-8 -*-
"""
3-class CNN evaluator (one/two/normal)
- Fixes applied:
  1) クラス順のズレ補正（学習順→ジェネレータ順マッピング）
  2) 学習と同一の前処理: 画像は RGB / (300,300) / 1./255
  3) 二重正規化の防止: ImageDataGenerator(rescale=1./255) のみ。追加の /255 はしない
  4) 入力チャネルをRGB固定（color_mode='rgb'）
  7) predict前に test_gen.reset() を実行し順序を固定
"""
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

# =========================
# パス設定（必要に応じて変更）
# =========================
TEST_DIR   = r"C:\path\to\data\test_cnn"  # フォルダ直下に one/ two/ normal/ の3フォルダ
MODEL_PATH = r"C:\path\to\data\models_cnn_traj\traj_cnn_3cls_xxx.h5"
OUT_ROOT   = r"C:\path\to\data\output"

CM_DIR   = os.path.join(OUT_ROOT, "confusion_matrix")
MISS_DIR = os.path.join(OUT_ROOT, "misclassified")
REPORT_DIR = os.path.join(OUT_ROOT, "report")
os.makedirs(CM_DIR, exist_ok=True)
os.makedirs(MISS_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

# 出力ファイル名に使う時刻
NOW = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# =========================
# 評価パラメータ（学習に準拠）
# =========================
IMG_SIZE   = (300, 300)  # 学習と同じ
BATCH_SIZE = 16
# ★学習時の出力クラス順（学習スクリプトの CLASSES と一致させる）
TRAIN_CLASSES = ["one", "two", "normal"]

# =========================
# 参考: GPU が見えているか（任意で確認）
# =========================
try:
    gpus = tf.config.list_physical_devices('GPU')
    print(f"[INFO] GPUs: {gpus}")
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
except Exception as e:
    print("[WARN] GPU 初期化メモリ成長設定に失敗:", e)

# =========================
# モデル読込
# =========================
print(f"[INFO] Loading model: {MODEL_PATH}")
model = load_model(MODEL_PATH)

# =========================
# テストジェネレータ（学習の前処理に合わせる）
# =========================
# 学習側は tf.image.resize + /255 のみ（Augなし）だったため、
# 評価も RGB / (300,300) / 1./255 のみを適用。色はRGB固定。
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    color_mode='rgb',         # ④ RGB固定
    interpolation='bilinear'  # tf.image.resize の既定（近似）に合わせる
)

# ジェネレータ側のクラス辞書（dir名→index）。※辞書順で index が決まる点に注意
gen_class_indices = test_gen.class_indices  # 例: {'normal':0,'one':1,'two':2}
# index→ラベル名 のリスト（ジェネレータ順）
gen_labels = [k for k, _ in sorted(gen_class_indices.items(), key=lambda kv: kv[1])]

print("[INFO] generator class_indices:", gen_class_indices)
print("[INFO] generator label order:", gen_labels)
print("[INFO] TRAIN_CLASSES (model output order):", TRAIN_CLASSES)

# =========================
# 1) 学習クラス順 → ジェネレータ順へのマッピングベクトルを作成
# =========================
# ここでズレを補正しないと、混同行列やレポートが壊滅的に悪化します。
try:
    map_model_to_gen = np.array([gen_class_indices[c] for c in TRAIN_CLASSES], dtype=int)
except KeyError as e:
    raise RuntimeError(
        f"学習時クラス {TRAIN_CLASSES} に含まれる '{e.args[0]}' が "
        f"テストディレクトリのクラス {list(gen_class_indices.keys())} に存在しません。"
    )

# =========================
# 予測（predict前に 7) reset を必ず実施）
# =========================
test_gen.reset()  # ⑦
y_prob_model = model.predict(test_gen, verbose=1)  # (N, C_model) ここでの列順は TRAIN_CLASSES
y_pred_model = np.argmax(y_prob_model, axis=1)     # モデル出力index（TRAIN_CLASSES系）

# --- 1) マッピングで“ジェネレータindex系”の予測に変換 ---
y_pred = map_model_to_gen[y_pred_model]            # これで y_true と同じ index 系列になる

# --- 正解（ジェネレータindex）とファイル相対パス ---
y_true = test_gen.classes
file_relpaths = test_gen.filenames

# =========================
# 混同行列（PNG）— ラベル順は“ジェネレータ順”
# =========================
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(gen_labels))))
cm_png = os.path.join(CM_DIR, f"confusion_matrix_3cls_{NOW}.png")

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=gen_labels, yticklabels=gen_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (3-class)")
plt.tight_layout()
plt.savefig(cm_png, dpi=150)
plt.close()
print(f"✅ 混同行列を保存しました: {cm_png}")

# =========================
# 分類レポート（ジェネレータ順で出力）
# =========================
report_txt = classification_report(
    y_true, y_pred,
    labels=list(range(len(gen_labels))),
    target_names=gen_labels,
    digits=4
)
report_path = os.path.join(REPORT_DIR, f"classification_report_3cls_{NOW}.txt")
with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_txt)
print(f"✅ 分類レポート保存: {report_path}")
print("\n[Classification Report]\n" + report_txt)

# =========================
# 誤分類一覧 CSV（True/Pred/各クラス確率）
#  確率も“ジェネレータ順”で列整列（可読性のため）
# =========================
# モデル出力確率（TRAIN_CLASSES順）→ ジェネレータ順に並べ替え
order_idx = [TRAIN_CLASSES.index(lbl) for lbl in gen_labels]  # ジェネ順で“モデル出力列の位置”を取る
y_prob_genorder = y_prob_model[:, order_idx]                  # (N, C) with columns in gen_labels order

df_all = pd.DataFrame({
    "file": [os.path.join(TEST_DIR, p) for p in file_relpaths],
    "true": [gen_labels[i] for i in y_true],
    "pred": [gen_labels[i] for i in y_pred],
})
for i, lbl in enumerate(gen_labels):
    df_all[f"p_{lbl}"] = y_prob_genorder[:, i]

df_miss = df_all[df_all["true"] != df_all["pred"]].copy()
miss_csv = os.path.join(MISS_DIR, f"misclassified_3cls_{NOW}.csv")
df_miss.to_csv(miss_csv, index=False, encoding="utf-8-sig")
print(f"✅ 誤分類CSVを保存しました: {miss_csv}")

# （必要なら全件CSVも保存）
# pred_all_csv = os.path.join(OUT_ROOT, f"predictions_3cls_{NOW}.csv")
# df_all.to_csv(pred_all_csv, index=False, encoding="utf-8-sig")
# print(f"ℹ️ 全件予測CSV: {pred_all_csv}")


[INFO] Loading model: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\models_cnn_traj\traj_cnn_3cls_20260112-041725.h5
Found 380 images belonging to 3 classes.
[INFO] class_indices (dir -> index): {'normal': 0, 'one': 1, 'two': 2}
[INFO] generator label order: ['normal', 'one', 'two']
[INFO] display/order for outputs: ['one', 'two', 'normal']
  7/380 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step  

c:\kanno\envs\MLya\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
✅ 混同行列を保存しました: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\confusion_matrix\confusion_matrix_20260112_162604.png

[Classification Report]
              precision    recall  f1-score   support

         one     0.0118    0.0167    0.0138        60
         two     0.0214    0.0543    0.0307        92
      normal     0.0164    0.0044    0.0069       228

    accuracy                         0.0184       380
   macro avg     0.0165    0.0251    0.0171       380
weighted avg     0.0169    0.0184    0.0138       380

✅ 誤分類CSVを保存しました: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\misclassified\misclassified_20260112_162604.csv
